In [1]:
#!/usr/bin/env julia
using Plots
using Yao
using Yao.ConstGate # needed for P1 = 0.5*(I - sigma_z) block
using StatsBase

In [2]:
# get_edges Revisa que puntos están a una distancia menor que 1
# y con eso arma las aristas los que están mas lejos, los pronde por defecto?
function get_edges(graph::Vector{NTuple{2, Float64}})
    Nv = size(graph)[1]
    edges = falses(Nv, Nv)
    for i in 1:(Nv-1)
        xi, yi = graph[i]
        for j in (i+1):Nv
            xj, yj = graph[j]

            dij = sqrt((xi - xj)^2. + (yi - yj)^2.)
            if dij <= 1.0
                edges[i,j] = true
            end
        end
    end
    return findall(edges)
end

function Ω(t::Float64)
    if 0 <= t <= 0.25
        return (Ω_max / 0.25) * t
    elseif 0.25 < t <= 0.69
        return Ω_max
    elseif 0.69 < t <= 1
        return - Ω_max * t / 0.31 + Ω_max * (1 + 0.69/0.31)
    end
end

function Ω2(t::Float64)
    aux=-8*(t-0.5)^2+2
    return aux
end

function δ(t::Float64)
    slope = (δ_0 - δ_max)/(0.25 - 0.69)
    if 0 <= t <= 0.25
        return δ_0
    elseif 0.25 < t <= 0.69
        return t * slope + (δ_max - slope * 0.69)
    elseif 0.69 < t <= 1
        return δ_max
    end
end

function  δ2(t::Float64)
    aux=2/(1 + exp(-10*(t-0.5)))-1
    return aux
end

function hamiltonian(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    #= H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j =#
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2]
        repeat(Nv,u*P1,(l,m))
    end |> sum
    interaction_term - δ(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω(t)*sum(map(i->put(Nv,i=>X), 1:Nv))
end

function hamiltonian2(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    #= H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j =#
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2]
        repeat(Nv,u*P1,(l,m))
    end |> sum
    interaction_term - δ2(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω2(t)*sum(map(i->put(Nv,i=>X), 1:Nv))
end

function run_annealing(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])
    for t in 0:dt:1.0
        h = hamiltonian(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt * 100)
    end
    return psi_t
end

function run_annealing2(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])
    for t in 0:dt:1.0
        h = hamiltonian2(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt * 100)
    end
    return psi_t
end

run_annealing2 (generic function with 1 method)

In [3]:
# Input Data
const u = 1.35
const Ω_max = 1.89
const δ_0 = -1.0
const δ_max = 1.0
dt = 0.001;

In [33]:
t=Array(LinRange(0., 1., 100))
y=Array([Ω(t[i]) for i = 1:100])
y2=Array([Ω2(t[i]) for i = 1:100])
plot(t,y,label = "Original Schedule")
plot!(t,y2,label = "Alternative Schedule")
plot!(xlabel = "t", ylabel = "Ω(t)")

In [34]:
using Plots
t=Array(LinRange(0., 1., 100))
y=Array([δ(t[i]) for i = 1:100])
y2=Array([δ2(t[i]) for i = 1:100])
plot(t,y,label = "Original Schedule")
plot!(t,y2,label = "Alternative Schedule")
plot!(xlabel = "t", ylabel = "δ(t)",legend=:bottomright)

### Solving  Problem 2 using Quantum Annealing 

In [22]:
graph = [(0.3461717838632017, 1.4984640297338632), 
         (0.6316400411846113, 2.5754677320579895), 
         (1.3906262250927481, 2.164978861396621), 
         (0.66436005100802, 0.6717919819739032), 
         (0.8663329771713457, 3.3876341010035995), 
         (1.1643107343501296, 1.0823066243402013)];

In [23]:
nshot=1000
edges = get_edges(graph);
psi = run_annealing(graph, edges, dt)
samples = measure(psi; nshots=nshot);

In [24]:
plotly()
Plots.PlotlyBackend()
a=countmap(samples)        # arma dic conf=>cuentas
confs=collect(keys(a))     # array de confs (Binario)
cuentas=collect(values(a)); # array de cuentas
#

cuentas2=[]
ticklabel = String[]
for k in 1:size(confs)[1]
    if  cuentas[k] > (nshot/10)
        zz=""
        for i in 1:6
            zz=zz*string(confs[k][i])
        end
        push!(ticklabel,zz)
        push!(cuentas2,cuentas[k])
    end
end
plot_ref=bar(cuentas2, xticks=(1:size(confs)[1], ticklabel), legend = false, title="Original Schedule")
xaxis!(xrotation=45)
#png(fn) # save the current fig as png with filename fn
#png(plot_ref, fn)

In [25]:
nshot=1000
edges = get_edges(graph);
psi = run_annealing2(graph, edges, dt)
samples = measure(psi; nshots=nshot);

In [26]:
plotly()
Plots.PlotlyBackend()
a=countmap(samples)        # arma dic conf=>cuentas
confs=collect(keys(a))     # array de confs (Binario)
cuentas=collect(values(a)); # array de cuentas
#
cuentas2=[]
ticklabel = String[]
for k in 1:size(confs)[1]
    if  cuentas[k] > (nshot/10)
        zz=""
        for i in 1:6
            zz=zz*string(confs[k][i])
        end
        push!(ticklabel,zz)
        push!(cuentas2,cuentas[k])
    end
end
bar(cuentas2, xticks=(1:size(confs)[1], ticklabel), legend = false, title="Alternative Schedule")
xaxis!(xrotation=45)

### Solving  Bruce Wayne's Problem using Quantum Annealing 

In [27]:
# Data
graph_gotham=[(1.19, 4.25),
              (2.71, 3.48),
              (1.19, 3.51),
              (2., 3.38),
              (1.12, 2.86),
              (1.70, 2.42),
              (2.36, 2.54),
              (1.52, 1.48),
              (2.15, 1.54),
              (2.14, 1.87),
              (1.72, 0.86),
              (2.29, 0.87)];

In [28]:
nshot=10000
edges = get_edges(graph_gotham);
psi = run_annealing(graph_gotham, edges, dt)
samples = measure(psi; nshots=nshot);

In [29]:
plotly()
Plots.PlotlyBackend()
a=countmap(samples)        # arma dic conf=>cuentas
confs=collect(keys(a))     # array de confs (Binario)
cuentas=collect(values(a)); # array de cuentas
#
cuentas2=[]
ticklabel = String[]
for k in 1:size(confs)[1]
    if  cuentas[k] > (nshot/40)
        zz=""
        for i in 1:12
            zz=zz*string(confs[k][i])
        end
        push!(ticklabel,zz)
        push!(cuentas2,cuentas[k])
    end
end
bar(cuentas2, xticks=(1:size(confs)[1], ticklabel), legend = false, title="Original Schedule")
xaxis!(xrotation=45)

In [30]:
nshot=10000
edges = get_edges(graph_gotham);
psi = run_annealing2(graph_gotham, edges, dt)
samples = measure(psi; nshots=nshot);

In [32]:
plotly()
Plots.PlotlyBackend()
a=countmap(samples)        # arma dic conf=>cuentas
confs=collect(keys(a))     # array de confs (Binario)
cuentas=collect(values(a)); # array de cuentas
#
cuentas2=[]
ticklabel = String[]
for k in 1:size(confs)[1]
    if  cuentas[k] > (nshot/40)
        zz=""
        for i in 1:12
            zz=zz*string(confs[k][i])
        end
        push!(ticklabel,zz)
        push!(cuentas2,cuentas[k])
    end
end
bar(cuentas2, xticks=(1:size(confs)[1], ticklabel), legend = false, title="Alternative Schedule")
xaxis!(xrotation=45)